[코드 돌릴 때 유의사항]
- 해당 코드는 어떤 과목이든 자연캠 리뷰만 조사합니다. 인문캠 과목이여도 자연캠 기준으로만 돌아갑니다.
- 코드 돌아갈 때 만약에 과목을 검색했을 때, 과목명이 나오지 않으면 바로 다음 과목으로 넘어가게끔 조치했습니다

In [1]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import requests
import pandas as pd
import time
import re
import random

In [2]:
lec_1 = pd.read_excel('교과목코드집_교양.xlsx')
lec_2 = pd.read_excel('교과목코드집_전공_대학원제거.xlsx')
merge_lec = pd.DataFrame()
merge_lec = merge_lec.append(lec_1)
merge_lec = merge_lec.append(lec_2)
merge_lec.reset_index(drop=True,inplace=True)

/var/folders/gx/q7vn3yv17rvb0vy6chbbbztw0000gn/T/ipykernel_11451/93745514.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merge_lec = merge_lec.append(lec_1)
/var/folders/gx/q7vn3yv17rvb0vy6chbbbztw0000gn/T/ipykernel_11451/93745514.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merge_lec = merge_lec.append(lec_2)


In [5]:
merge_lec

,교과목명,학점수,강좌유형,NP유형,이수학년,학과
0,성서와인간이해,2,이론,NaN,전학년,교양
1,채플,0,이론,채플,전학년,교양
2,현대사회와기독교윤리,2,이론,NaN,전학년,교양
3,종교와과학,2,이론,NaN,전학년,교양
4,글쓰기,3,이론,NaN,전학년,교양
...,...,...,...,...,...,...
3484,공공디자인,3,이론 및 실습,NaN,4학년,도경
3485,도시경관디자인워크숍,3,이론 및 실습,NaN,3학년,도경
3486,창업현장실습,3,실습,전공NP,전학년,창업
3487,기능성식품,3,이론,NaN,4학년,식영


In [16]:
# merge_lec으로 얻을 수 있는 정보
lecturelist = [] # 교과목
creditlist = [] # 학점
classification_list=[] # 이론 실습 강좌유형
pnplist = [] # pass or np
gradelist = [] # 이수학년
majoryon = [] # 전공인지아닌지
majorlist = [] # 학과칼럼

# 크롤링해서 얻어야 하는 정보
proflist = [] # 교수명
reviewlist = [] # 리뷰 목록
whenlist = [] # 작성시점
scorelist = [] # 평가별 점수
votelist = [] # 공감 수

## 로그인해서 강의실 페이지 들어가기까지

In [17]:
driver = webdriver.Chrome(executable_path = 'C:/Users/chromedriver.exe')
driver.implicitly_wait(3)  # 로딩 될때까지 3초 대기
# 3초 대기
time.sleep(3)

driver.get("https://everytime.kr/lecture")
time.sleep(2)

id_bar = driver.find_element(By.CSS_SELECTOR, '#container > form > p:nth-child(1) > input')
id_bar.clear()
id_bar.send_keys('oneman98')
time.sleep(2)



pw_bar = driver.find_element(By.CSS_SELECTOR,'#container > form > p:nth-child(2) > input')
pw_bar.clear()
#비밀번호 입력해야함
pw_bar.send_keys('sinhang789@')

pw_bar.send_keys(Keys.RETURN)
time.sleep(3)

/var/folders/gx/q7vn3yv17rvb0vy6chbbbztw0000gn/T/ipykernel_11451/258489869.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = 'C:/Users/chromedriver.exe')


## 모든 과목 검색 후 강의 관련 데이터 크롤링

In [18]:
# 랜덤한 대기시간
wait_sec = (random.random()*10)%3+1

for i in range(1215, len(merge_lec)):
    # 검색 바 찾아서 검색
    search_bar = driver.find_element(By.CSS_SELECTOR, '#container > form > input.keyword')
    search_bar.clear()
    # 과목명 입력
    search_bar.send_keys(merge_lec['교과목명'][i])
    search_bar.send_keys(Keys.RETURN)
    time.sleep(2)
    
    # 자연캠 버튼 클릭
    button = driver.find_element(By.CLASS_NAME, 'campus')
    button.click()
    choice = driver.find_element(By.CSS_SELECTOR,'body > div > div.dialogmenu > div.wrap > div:nth-child(2) > div')
    choice.click()
    
    # 과목명
    subject = merge_lec['교과목명'][i]
    print(subject + str(i))
    credit = merge_lec['학점수'][i]
    classification = merge_lec['강좌유형'][i]
    # 패논패과목이면 o, 아니면 x로 구분
    cat = merge_lec['NP유형'][i]
    if cat=='채플' or cat=='교양NP' or cat=='사회봉사' or cat=='진로선택' or cat=='전공NP':
        pnp = "o"
    else:
        pnp = "x"
    grade = merge_lec['이수학년'][i]
    
    # 교양이라고 적혀 있으면 교양, 아니면 전공
    # ismajor : 교양인지 전공인지
    # major : 교양인지 전공인지, 단, 전공이면 어떤 과의 전공인지까지 나와있음
    if merge_lec['학과'][i] == "교양":
        ismajor = '교양'
        major = merge_lec['학과'][i]
    else:
        ismajor = "전공"
        major = merge_lec['학과'][i]
        
    # 끝까지 내리기
    a=0
    while a < 10000:
        a = a + 1
        driver.execute_script("window.scrollTo(0, 100100100100100100100100);")

    # 끝까지 올리기
    a=0
    while a < 10000:
        a = a + 1
        driver.execute_script("window.scrollTo(0, -100100100100100100100100);")
        driver.implicitly_wait(2)
        
    # 과목명 검색해서 찾기
    lectures = driver.find_elements(By.CLASS_NAME, 'lecture')
    if len(lectures)!=0:
        # 인문캠 기준으로 과목명이 존재하는지 존재하지 않는지 비교
        # 과목명이 존재하지 않는다면, 생략하고 바로 다음으로

        # 인문캠부터 수행
        for lecture in lectures:
            # 강의명, 교수명 저장
            try : 
                prof = lecture.text.split('\n')[1]
            except : 
                prof = "없음"
            # 강의 클릭하기
            lecture.click()
            # 강의평 더보기 클릭
            # 만약 강의평이 없다면, 뒤의 과정 수행은 중지하고 다음 과정으로 넘어갈 수 있게끔
            try:
                driver.find_element(By.CSS_SELECTOR, 'body > div > div > div.pane > div > section.review > div.articles > a').click()
                time.sleep(wait_sec)
            except : 
                driver.back()
                time.sleep(wait_sec+0.2)
                continue # 바로 다음 항목으로 넘어가게끔

            # 강의평 끝까지 내리기
            itemlist = driver.find_element(By.CLASS_NAME, "articles")
            b=0
            while b<10000:
                b = b + 1
                driver.execute_script("arguments[0].scrollBy(0, 100100100100100100100100)", itemlist)

            # 리뷰 담기
            reviews = driver.find_elements(By.CLASS_NAME, 'text')
            for review in reviews:
                reviewlist.append(review.text)

            # 수강학기 담기 + 교수명, 강의명 담기
            whens = driver.find_elements(By.CLASS_NAME, 'semester')

            for when in whens:
                lecturelist.append(subject) # 과목명 담기
                creditlist.append(credit) # 학점수 담기
                classification_list.append(classification) # 이론실습 강좌유형 담기
                pnplist.append(pnp) # 패논패여부 담기
                gradelist.append(grade) # 이수학년 담기
                majoryon.append(ismajor) # 전공여부
                majorlist.append(major) # 교양, 전공이면 세부과 담기

                proflist.append(prof) # 교수명 담기
                whenlist.append(when.text) # 수강학기 담기


            # 별 담기
            bs = BeautifulSoup(driver.page_source, 'html.parser')
            # # 별 개수
            bs_stars = bs.find_all('span', class_='on')
            # 1부터 한 이유는, 첫번째 즉, 0의 결과가 전체 평점이 나오기 때문
            for i in range(1, len(bs_stars)):
                score = bs_stars[i]['style'].split(': ')[1].replace(';','')
                # score에 따른 점수 할당 후 담기
                if score == '0%':
                    LecStar = 0
                    scorelist.append(LecStar)
                elif score == '20%':
                    LecStar = 1
                    scorelist.append(LecStar)
                elif score == '40%':
                    LecStar = 2
                    scorelist.append(LecStar)
                elif score == '60%':
                    LecStar = 3
                    scorelist.append(LecStar)
                elif score == '80%':
                    LecStar = 4
                    scorelist.append(LecStar)
                elif score == '100%':
                    LecStar = 5
                    scorelist.append(LecStar)

            # 추천받은 수
            votes = driver.find_elements(By.CLASS_NAME, 'info')

            for i in range(1, len(votes)):
                try:
                    vote = votes[i].text.split('\n')[1]
                    vote = int(vote)
                    votelist.append(vote)
                except:
                    vote = 0
                    votelist.append(vote)

            ## 뒤로가기 두번 누르면 원래 강의평가 페이지로 넘어가짐
            driver.back()
            time.sleep(wait_sec+0.2)
            driver.back()
            time.sleep(wait_sec+0.5)# 크롤링 감지 방지를 위해 sleep시간 다르게 주기
    else:
        time.sleep(1)
    driver.back()
    time.sleep(wait_sec+0.3)

인터넷과커뮤니케이션150
경상통계학151
현대사회와정보152
행정학개론153
인간과정치154
인간관계와커뮤니케이션155
미시경제학원론156
거시경제학원론157
청소년지도학158
지역사회복지론159
사회복지실천론160
기업과경영161
사회복지개론162
인간행동과사회환경163
경제학원론164
시장경제와법165
직업생활과법166
생활법률167
법률문장론168
지적재산과현대사회169
법과제도170
창업현장실습171
창업현장실습172
다문화사회의이해173
ICT비즈니스와경영174
마케팅과ICT융합기술175
저작권과소프트웨어176
창업현장실습177
경제학들어가기178
영화와법179
다문화간의커뮤니케이션180
明청년심리학181
유라시아의이해182
북한의이해183
모의벤처창업184
아시아의국제관계185
아시아에서의글로벌리더십186
글로벌경영과마케팅187
아시아문화관광경영188
창의적비즈니스189
창업현장실습190
미래사회와경영191
소셜벤처의이해192
소프트웨어시장및창업193
창의적성격과사고기법194
사회변동과가족의미래195
워크숍중심의행복한예비부모교육196
한중일관계사의비극197
민주시민과청년의삶198
MJ사회봉사199
현대한국정치의쟁점200
소셜미디어와스마트한세상(KCU)201
4차산업혁명시대의창의적사고202
놀이혁명과창의성203
현대세계의분쟁과평화204
제도와경제205
세계화,시장과민주주의(KCU)206
미국사회와정치207
디지털시대인공지능(A.I.)과법208
쉽게배우는글로벌기업전략209
글로벌마케팅경영입문210
통합적커리어탐색:취업과창업211
미리알아두면힘이되는노동인권212
NCS의이해와공기업취업전략213
지구촌이슈와함께하는세계시민교육214
진로와인간관계를위한이미지메이킹215
플랫폼시대사회적기업과공유경제216
시장경제와기업217
직무커뮤니케이션능력개발218
금융기관의이해219
창업사례연구220
창업시뮬레이션221
창업현장실습222
창업현장실습223
창업현장실습224
창업현장실습225
창업현장실습226
창업현장실습227
미적분학1228
미적분학2229
공학수학1230


KeyboardInterrupt: 

In [19]:
print(len(lecturelist))
print(len(creditlist))
print(len(classification_list))
print(len(pnplist))
print(len(gradelist))
print(len(majoryon))
print(len(majorlist))
print(len(proflist))
print(len(reviewlist))
print(len(whenlist))
print(len(scorelist))
print(len(votelist))

3079
3079
3079
3079
3079
3079
3079
3079
3079
3079
3079
3079


In [20]:
df = pd.DataFrame(zip(lecturelist, creditlist, classification_list, pnplist, gradelist, majoryon, majorlist, proflist, reviewlist, whenlist, scorelist, votelist))

df.columns = ['강의명','학점','강좌유형','패논패여부','이수학년','전공여부','학과','교수명','리뷰','작성시점','평가별점수','공감수']

In [21]:
df['강의명'].unique()

array(['인간관계와커뮤니케이션', '생활법률', '다문화사회의이해', '워크숍중심의행복한예비부모교육',
       'NCS의이해와공기업취업전략', '미적분학1', '미적분학2', '공학수학1', '공학수학2', '통계학개론',
       '선형대수학개론', '이산수학개론', '수치해석', '물리학1', '물리학2', '물리학실험1', '물리학실험2',
       '일반생물학', '생물학1', '생물학2', '생물학실험1', '생물학실험2', '일반화학', '일반화학실험',
       '화학1', '화학2', '화학실험1', '화학실험2', '고급이산수학', '응용통계학', '일반생물학실험',
       '미적분학연습1', '수학콘서트', '산업안전', '기술과경영', '창의공학', '전공자유학부세미나', '산업경영',
       '게임학개론', '체육사', '스포츠맛사지', '게임과인간심리', '스포츠와리더십', '스포츠와체육의탐구',
       '교양바둑', '운동생리의이해', '식품과영양의이해', '여가와스포츠관광', '중국어입문1', '토익(RC)',
       '토익(LC)', 'C언어', 'CAD개론', 'C++Programming', '엑셀기초및실무활용',
       '웹프로그래밍기초', '디지털상상력과이미지활용', '웹과앱기초', '파이썬프로그래밍입문', '기독교와합창',
       '기독교와리더쉽', '기독교와성결혼가정', '윤리학에로초대', '영화로보는역사', '영화와철학', '한국의역사와문화',
       '미학의이해', '지중해문화여행', '예술과디자인', '서양음악의이해', '국악의이해', '사진영상미디어의이해',
       '금융시장의이해', '현대가족과부모', '현대중국의이해', '현대일본의이해', '글로벌리더를위한글로벌매너',
       '진로선택과대학생활', '중국역사와문화', '자기분석과직무역량개발', '자기경영과실전취업준비',
       '소비자를위한생활법률(KCU)', '지구에서만나살아가는남자와여

- 대중문화와 매스컴까지 돌렸고 나머지는 집가서 돌릴예정

In [22]:
df.to_csv('자연캠전과목리뷰(1215~).csv', index=False, encoding='utf-8-sig')

In [40]:
pd.read_csv('자연캠전과목리뷰(대중문화와매스컴이후).csv')

,강의명,학점,강좌유형,패논패여부,이수학년,전공여부,학과,교수명,리뷰,작성시점,평가별점수,공감수
0,인간관계와커뮤니케이션,3,이론,x,전학년,교양,교양,미배정,처음에 강의명보고 인간관계에 대한 심리? 에대해 배우는줄알았는데... 그냥 전형적인...,19년 2학기 수강자,3,0
1,인간관계와커뮤니케이션,3,이론,x,전학년,교양,교양,강재호,일단 이번학기는 사강이었기 때문에 조별과제같은건 없고 출결도 쉬웠습니다. 과제는 간...,20년 1학기 수강자,4,1
2,인간관계와커뮤니케이션,3,이론,x,전학년,교양,교양,강재호,수업이 세시간연강이었던터라 지루하긴 했지만 시험공부하는데 있어서는 매우 수월함,18년 2학기 수강자,3,0
3,인간관계와커뮤니케이션,3,이론,x,전학년,교양,교양,강재호,시험직전에 찍어주신거만 하면 a에서 a+가능,19년 1학기 수강자,4,0
4,인간관계와커뮤니케이션,3,이론,x,전학년,교양,교양,강재호,"출튀 많이해도 시험 범위만 알고 공부하면 바닥은 면하는 수업이지만, 출튀를 많이 하...",19년 1학기 수강자,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3074,영어회화,3,실습,x,2학년,전공,영문,에릭스톨즈,최고의 교수님\n두 학기 연속으로 들었는데 학점 정말 잘 주신다.\n항상 학생들을 ...,22년 1학기 수강자,5,0
3075,영어회화,3,실습,x,2학년,전공,영문,에릭스톨즈,"학점 절대평가인 것 같고, 비대면이라 좋았음\n에릭 교수님 이번 학기가 마지막이라 ...",22년 1학기 수강자,5,0
3076,영어회화,3,실습,x,2학년,전공,영문,에릭스톨즈,에릭교수님 점수 후하게 잘 주시고 좋습니다!!,22년 1학기 수강자,5,0
3077,영어회화,3,실습,x,2학년,전공,영문,에릭스톨즈,영회는 에릭 교수님이 짱이다. 진심으로 영어를 못해도 잘 이끌어주시고 점점 영어회화...,22년 1학기 수강자,5,1


In [23]:
merge_lec[merge_lec['교과목명']=='영어회화']

,교과목명,학점수,강좌유형,NP유형,이수학년,학과
513,영어회화,3,실습,NaN,2학년,영문


In [25]:
merge_lec['학과'].unique()

array(['교양', '국문', '영문', '중문', '일문', '사학', '사행', '문정', '아랍', '청지', '철학',
       '문창', '아동', '미사', '국한', '국글', '한아', '글한', '행정', '정외', '북한', '경제',
       '미디', '사복', '보상', '경상', '경과', '경보', '경영', '경정', '통상', '부동', '법학',
       '기록', '법무', '디콘', '융소', '응소', '빅데', '데테', '미용', '웹툰', '물류', '뮤콘',
       '뷰경', '아상', '아심', '유보', '융예', '미경', '심리', '만화', '복경', '복상', '아복',
       '유경', '멀티', '스포', '융디', '아경', '미래', '복지', '프롭', '혁신', '수학', '물리',
       '화학', '식영', '생명', '공과', '융합', '창업', '전기', '전자', '정통', '화공', '소재',
       '환경', '토환', '토목', '교통', '컴공', '산업', '기계', '반도', '영상', '체학', '체육',
       '사체', '스산', '경기', '바둑', '성악', '피아', '작곡', '뮤지', '영화', '디자', '산자',
       '시각', '패션', '건축', '건학', '전건', '공간', '국제', '도경'], dtype=object)

In [39]:
merge_lec[merge_lec['학과']=='보상']

,교과목명,학점수,강좌유형,NP유형,이수학년,학과
1215,전자상거래,3,이론,NaN,전학년,보상


In [29]:
merge_lec[merge_lec['학과']=='미용']

,교과목명,학점수,강좌유형,NP유형,이수학년,학과
1635,뷰티매니지먼트,3,이론,NaN,3학년,미용
1636,어드밴스드메이크업,3,이론 및 실습,NaN,3학년,미용
1637,뷰티업스타일링,3,이론 및 실습,NaN,3학년,미용
1638,크리에이티브퍼머넌트,3,이론 및 실습,NaN,3학년,미용
1639,퍼스널컬러및이미지메이킹,3,이론,NaN,3학년,미용
1640,크리에이티브커트,3,이론 및 실습,NaN,3학년,미용
1641,미용사진영상학,3,이론,NaN,3학년,미용
1642,뷰티디자인발상,3,이론,NaN,4학년,미용
1643,크리에이티브헤어아트,3,이론 및 실습,NaN,4학년,미용
1644,미용프레젠테이션,3,이론,NaN,4학년,미용


In [31]:
len(merge_lec)

3489

In [33]:
merge_lec['학과'].unique()

array(['교양', '국문', '영문', '중문', '일문', '사학', '사행', '문정', '아랍', '청지', '철학',
       '문창', '아동', '미사', '국한', '국글', '한아', '글한', '행정', '정외', '북한', '경제',
       '미디', '사복', '보상', '경상', '경과', '경보', '경영', '경정', '통상', '부동', '법학',
       '기록', '법무', '디콘', '융소', '응소', '빅데', '데테', '미용', '웹툰', '물류', '뮤콘',
       '뷰경', '아상', '아심', '유보', '융예', '미경', '심리', '만화', '복경', '복상', '아복',
       '유경', '멀티', '스포', '융디', '아경', '미래', '복지', '프롭', '혁신', '수학', '물리',
       '화학', '식영', '생명', '공과', '융합', '창업', '전기', '전자', '정통', '화공', '소재',
       '환경', '토환', '토목', '교통', '컴공', '산업', '기계', '반도', '영상', '체학', '체육',
       '사체', '스산', '경기', '바둑', '성악', '피아', '작곡', '뮤지', '영화', '디자', '산자',
       '시각', '패션', '건축', '건학', '전건', '공간', '국제', '도경'], dtype=object)

In [ ]:
1215~2350
2351~